<a href="https://colab.research.google.com/github/vedanth-aggarwal/neo4j_knowledgegraph_cypherag/blob/main/RAG_With_Knowledge_graph(Neo4j).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.7/202.7 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.5/293.5 kB 27.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4

In [ ]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [ ]:
from google.colab import userdata
OPENAI_API_KEY= "" #userdata.get('OPENAI_API_KEY')

In [ ]:
from typing import Tuple, List, Optional

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser

In [ ]:
from langchain_core.runnables import ConfigurableField

In [ ]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase


In [ ]:
import os

In [ ]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [ ]:
from langchain_community.vectorstores import Neo4jVector

In [ ]:
from google.colab import userdata
OPENAI_API_KEY="" #userdata.get('OPENAI_API_KEY')

In [ ]:
'''
# Wait 60 seconds before connecting using these details, or login to https://console.neo4j.io to validate the Aura Instance is available
NEO4J_URI=neo4j+s://2a2c4c30.databases.neo4j.io
NEO4J_USERNAME=neo4j
NEO4J_PASSWORD=mVm-uSfheC4AsM_RRjgC9DK8W2fJOCflM3sA9VrbpaE
AURA_INSTANCEID=2a2c4c30
AURA_INSTANCENAME=Instance01
'''
NEO4J_URI="neo4j+s://2a2c4c30.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="mVm-uSfheC4AsM_RRjgC9DK8W2fJOCflM3sA9VrbpaE"

In [ ]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [ ]:
from langchain_community.graphs import Neo4jGraph

In [ ]:
graph = Neo4jGraph()

In [ ]:
from langchain.document_loaders import TextLoader

# Load documents from text files
raw_documents = []
for filename in ['10th century.txt', 'beer.txt']:
    loader = TextLoader(file_path=filename)
    raw_documents.extend(loader.load())

In [ ]:
from langchain.document_loaders import WikipediaLoader
raw_documents = WikipediaLoader(query="Elizabeth I").load()

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [ ]:
len(raw_documents)

2

In [ ]:
raw_documents[:3]

[Document(metadata={'title': 'Elizabeth I', 'summary': 'Elizabeth I (7 September 1533 – 24 March 1603) was Queen of England and Ireland from 17 November 1558 until her death in 1603. She was the last monarch of the House of Tudor.\nElizabeth was the only surviving child of Henry VIII and his second wife, Anne Boleyn. When Elizabeth was two years old, her parents\' marriage was annulled, her mother was executed, and Elizabeth was declared illegitimate. Henry restored her to the line of succession when she was 10, via the Third Succession Act 1543. After Henry\'s death in 1547, Elizabeth\'s younger half-brother Edward VI ruled until his own death in 1553, bequeathing the crown to a Protestant cousin, Lady Jane Grey, and ignoring the claims of his two half-sisters, the Catholic Mary and the younger Elizabeth, in spite of statutes to the contrary. Edward\'s will was set aside within weeks of his death and Mary became queen, deposing and executing Jane. During Mary\'s reign, Elizabeth was i

In [ ]:
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents)
print(len(documents))

19


In [ ]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo') #"gpt-3.5-turbo-0125")

In [ ]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)

In [ ]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [ ]:
graph_documents

[GraphDocument(nodes=[Node(id='10Th Century', type='Time period'), Node(id='European History', type='History'), Node(id='China', type='Country'), Node(id='Political Upheaval', type='Event'), Node(id='Islamdom', type='Culture'), Node(id='Spain', type='Country'), Node(id='Lynn White', type='Person'), Node(id='Helen Waddell', type='Person'), Node(id='Lorenzo Valla', type='Person'), Node(id='Yogini Goddess', type='Artifact'), Node(id='Kaveripakkam', type='Place'), Node(id='Tamil Nadu', type='Place'), Node(id='Medieval Warm Period', type='Event'), Node(id='Viking Groups', type='Group'), Node(id='Normans', type='Group'), Node(id='Cluny', type='Organization'), Node(id='Magyar Cavalry', type='Group'), Node(id='Mieszko I', type='Person'), Node(id='Poland', type='Country'), Node(id='Khazar Kingdom', type='Kingdom'), Node(id='Kievan Rus', type='Country')], relationships=[Relationship(source=Node(id='10Th Century', type='Time period'), target=Node(id='European History', type='History'), type='RELE

In [ ]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [ ]:
# directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

In [ ]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

In [ ]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [ ]:
def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    display(widget)
    return widget

In [ ]:
showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

GraphWidget(layout=Layout(height='800px', width='100%'))

In [ ]:
from typing import Tuple, List, Optional

In [ ]:
from langchain_community.vectorstores import Neo4jVector

In [ ]:
from langchain_openai import OpenAIEmbeddings
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [ ]:
vector_index.similarity_search("Who is Elizabeth I?")

[Document(metadata={'source': '10th century.txt'}, page_content='\ntext:  a Christian in 966\n     * Khazar kingdom is attacked and defeated by Kievan Rus ( 965)\n     * Vladimir I, Prince of Kievan Rus, baptised a Christian in 988\n     * Collapse of the central lowland Maya civilization. End of Classic\n       Maya period, begin Post-Classic Maya.\n     * Rise of the Toltecs in Mexico.\n     * Collapse of Great Moravia\n     * Buddhist temple construction commences at Bagan, Myanmar\n     * Laguna Copperplate Inscription, Kavi script, inscribed in Luzon,\n       Philippines, dated Saka year 822 (AD 900).\n     * The medieval Croatian state becomes a unified kingdom under\n       Tomislav\n     * Twentieth century Belgian astronomer Jean Meeus asserts that the\n       orbits of all nine planets were within the same 90% arc of the\n       solar system on 1 February 949. The next time it is thought this\n       will occur is on 6 May 1492.\n     * Coastal cities on the Malay Peninsula a

In [ ]:
graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

[]

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field
# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

In [ ]:
entity_chain = prompt | llm.with_structured_output(Entities)

In [ ]:
entity_chain.invoke({"question": "When was Beer invented in 10th century"}).names

['Beer', '10th century']

In [ ]:
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

In [ ]:
def generate_full_text_query(input: str) -> str:
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

generate_full_text_query('Amerlia Earhart')

'Amerlia~2 AND Earhart~2'

In [ ]:
# Fulltext index query
def structured_retriever(question: str) -> str:
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [ ]:
print(structured_retriever("Was beer invented in 10th century?"))

Beer - NOTED_BY -> Michael Jackson
Beer - ORIGINATED_IN -> Ancient Egypt
Beer - ORIGINATED_IN -> Mesopotamia
Beer - PRODUCTION_LOCATION -> Europe
Beer - PRODUCED_BY -> Monasteries
Beer - ADDING -> Sugar
Beer - ADDING -> Wort
Beer - CONTAINING -> Carbon Dioxide
Beer - CONTAINING -> Alcohol
Beer - SETTLING -> Yeast
Beer - SETTLING -> Trub
Beer - INCREASING -> Co2
Beer - PRIMED_IN -> Bottle
Beer - STORED_IN -> Keg
Beer - STORED_IN -> Cask
Beer - STORED_IN -> Bottles
Beer - HAS -> Dark Brew
Beer - CONTAINS -> Alcohol
Beer - CONTAINS -> Nutrients
Beer - ASSOCIATED_WITH -> Social Traditions
Beer - ASSOCIATED_WITH -> Rating
Beer - CONSUMED_IN -> Various Societies
Beer - DRUNK_FROM -> Vessels
Hops - ADDED_TO -> Beer
Sugar - ADDED_TO -> Beer
Flavouring - ADDED_TO -> Beer
Malted Barley - SOURCE_OF -> Beer
Yeast - COMPONENT_OF -> Beer
Water - COMPONENT_OF -> Beer
Starch Source - COMPONENT_OF -> Beer
People - SWAP -> Beer
Indonesia - MADE_IN -> Brem10Th Century - RELEVANT -> European History
10Th 

In [ ]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    #print(structured_data,end='\n\n')
    unstructured_data = [el.page_content[:50] for el in vector_index.similarity_search(question)]
    #print(unstructured_data)
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
"""

    #print(final_data)
    return final_data
#

In [ ]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

In [ ]:
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [ ]:
def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

In [ ]:
_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatOpenAI(temperature=0)
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

In [ ]:
template2 = """Answer the question based only on the following context:
{context}

Question: {question}

Answer:"""

template = """Answer the question based only on the following context:
{context}

Question: {question}
-> Give Answer 1 using only unstructured data
-> Give Answer 2 using only structured data
-> Give Answer 3 using both structured and unstructured data
-> Evaluate and state which answer is best
Answer1:
Answer2:
Answer3:
Evaluation:
"""

template1 = """Write a 300 word essay divided into 2 pargaphs on the prompt only using the context:
{context}

Prompt: {question}
Answer:"""

#Use natural language and be concise.

In [ ]:
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
print(chain.invoke({"question": "Did 10th century have any influence on beer"}))

Search query: Did 10th century have any influence on beer
Answer1: Yes, the unstructured data mentions the 10th century in relation to beer.

Answer2: No, the structured data does not provide any direct information about the influence of the 10th century on beer.

Answer3: Yes, the unstructured data mentions the 10th century in relation to beer, and the structured data provides information about the production, ingredients, and social traditions associated with beer during that time period.

Evaluation: Answer 3 is the best as it combines information from both structured and unstructured data to provide a comprehensive answer.


In [ ]:
chain.invoke(
    {
        "question": "When was she born?",
        "chat_history": [("Which house did Elizabeth I belong to?", "House Of Tudor")],
    }
)

Search query: When was Elizabeth I born?


'Elizabeth I was born on 7 September 1533.'